In [ ]:
import torch
import torch.nn as nn
import numpy as np
from transformers import BertTokenizer
import pickle

from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker

In [ ]:
import torch
from torch.nn import CrossEntropyLoss
import textattack
from model_wrapper import ModelWrapper

from models import model_1 as mm

In [ ]:
torch.cuda.empty_cache()

### PyTorchModelWrapper

In [ ]:
class PyTorchModelWrapper(ModelWrapper):
    """Loads a PyTorch model (`nn.Module`) and tokenizer.

    Args:
        model (torch.nn.Module): PyTorch model
        tokenizer: tokenizer whose output can be packed as a tensor and passed to the model.
            No type requirement, but most have `tokenizer` method that accepts list of strings.
    """

    def __init__(self, model, tokenizer):
        if not isinstance(model, torch.nn.Module):
            raise TypeError(
                f"PyTorch model must be torch.nn.Module, got type {type(model)}"
            )

        self.model = model
        self.tokenizer = tokenizer

    def to(self, device):
        self.model.to(device)

    def __call__(self, text_input_list, batch_size=32):
        model_device = next(self.model.parameters()).device
        ############################################################ Change 1
        ids = self.tokenizer(text_input_list, return_tensors="pt", padding=True)
        ids = ids.to(model_device)

        with torch.no_grad():
            outputs = textattack.shared.utils.batch_model_predict(
                self.model, ids, batch_size=batch_size
            )

        return outputs

    def get_grad(self, text_input, loss_fn=CrossEntropyLoss()):
        """Get gradient of loss with respect to input tokens.

        Args:
            text_input (str): input string
            loss_fn (torch.nn.Module): loss function. Default is `torch.nn.CrossEntropyLoss`
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """

        if not hasattr(self.model, "get_input_embeddings"):
            raise AttributeError(
                f"{type(self.model)} must have method `get_input_embeddings` that returns `torch.nn.Embedding` object that represents input embedding layer"
            )
        if not isinstance(loss_fn, torch.nn.Module):
            raise ValueError("Loss function must be of type `torch.nn.Module`.")

        self.model.train()

        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        
        ######################################################### Change 2
        ids = self.tokenizer(text_input_list, return_tensors="pt", padding=True)
        ids = ids.to(model_device)

        predictions = self.model(ids)

        output = predictions.argmax(dim=1)
        loss = loss_fn(predictions, output)
        loss.backward()

        # grad w.r.t to word embeddings

        # Fix for Issue #601

        # Check if gradient has shape [max_sequence,1,_] ( when model input in transpose of input sequence)

        if emb_grads[0].shape[1] == 1:
            grad = torch.transpose(emb_grads[0], 0, 1)[0].cpu().numpy()
        else:
            # gradient has shape [1,max_sequence,_]
            grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": ids[0].tolist(), "gradient": grad}

        return output

    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [self.tokenizer.convert_ids_to_tokens(self.tokenizer(x)) for x in inputs]

### Self Tx

In [67]:
class ad_transformer(nn.Module):
  def __init__(self, d_model, nhead, nlayer, max_token, embedding, device = "cuda"):
    super().__init__()

    self.d_model = d_model
    self.max_token = max_token

    self.embedding = torch.tensor(embedding)

    encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, batch_first=True, device = device)
    self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=nlayer).to("cuda")
    self.linear_out = nn.Linear(d_model * max_token, 2)

  def forward(self, input_str, device = "cuda"):

    #if type(input_str) != list:
    #    raise TypeError("Self Error : Invalid Input !!!")
      
    input = input_str["input_ids"]

    ## Tokens to embeding
    input_embd = torch.zeros(len(input), self.max_token, self.d_model)
    
    input_embd = input_embd + self.embedding[0]   ### Padding
      
    for batch_num in range(len(input)):
        for i in range(len(input[batch_num])):
            input_embd[batch_num][i] = self.embedding[input[batch_num][i]]
    input_embd = input_embd.to(device)
    
    out1 = self.transformer_encoder(input_embd)
    return self.linear_out(out1.reshape(len(out1), self.max_token * self.d_model))

In [86]:
d_model = 768
max_token = 50
nhead = 12
nlayer = 6
device = "cuda"

In [87]:
with open("bert_embedding.pkl", "rb") as file:
    embd = pickle.load(file)

In [88]:
model = ad_transformer(d_model,nhead, nlayer, max_token, embd, device).to(device)

In [89]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [90]:
enc_input = tokenizer.batch_encode_plus(["Hi", "Bye"], return_tensors="pt").to(device)

In [91]:
enc_input['input_ids']

tensor([[ 101, 7632,  102],
        [ 101, 9061,  102]], device='cuda:0')

In [92]:
model(enc_input, device)

tensor([[ 0.0471,  0.3120],
        [-0.0164,  0.3773]], device='cuda:0', grad_fn=<AddmmBackward0>)

### Attack

In [93]:
wrapper = PyTorchModelWrapper(model, tokenizer)

In [94]:
#dataset = HuggingFaceDataset("sms_spam",dataset_columns=["text","label"], shuffle=True)
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
attack = PWWSRen2019.build(wrapper)

textattack: Loading datasets dataset rotten_tomatoes, split test.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/aamod_thakur/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class '__main__.ad_transformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [95]:
dataset[0]

(OrderedDict([('text',
               'lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .')]),
 1)

In [96]:
attacker = Attacker(attack, dataset)

In [97]:
attacker.attack_dataset()

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 1 / 1 / 2:  20%|██████████████                                                        | 2/10 [00:00<00:00, 22.45it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (60%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (62%)]] --> [[[FAILED]]]

consistently clever and suspenseful .




[Succeeded / Failed / Skipped / Total] 1 / 1 / 2 / 4:  40%|████████████████████████████                                          | 4/10 [00:00<00:00,  8.37it/s]

--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (58%)]] --> [[Negative (53%)]]

it's like a " big chill " reunion of the baader-meinhof gang , only these [[guys]] are more harmless [[pranksters]] than political activists .

it's like a " big chill " reunion of the baader-meinhof gang , only these [[rib]] are more harmless [[cut-up]] than political activists .


--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (56%)]] --> [[[SKIPPED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .




...

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (59%)]] --> [[Negative (66%)]]

fresnadillo has something serious to say about the ways in which extravagant chance can [[distort]] our perspective and throw us off the path of good sense .

fresnadillo has something serious to say about the ways in which extravagant chance can [[wring]] our perspective and throw us off the path of good sense .




[Succeeded / Failed / Skipped / Total] 5 / 1 / 3 / 9:  90%|███████████████████████████████████████████████████████████████       | 9/10 [00:01<00:00,  4.91it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (59%)]] --> [[Negative (55%)]]

[[throws]] in enough clever and unexpected twists to make the formula feel fresh .

[[bedevil]] in enough clever and unexpected twists to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (68%)]] --> [[Negative (52%)]]

weighty and [[ponderous]] but every bit as filling as the treat of the title .

weighty and [[heavy]] but every bit as filling as the treat of the title .


--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (59%)]] --> [[[SKIPPED]]]

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .




[Succeeded / Failed / Skipped / Total] 5 / 2 / 3 / 10: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  5.00it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Positive (61%)]] --> [[[FAILED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 3      |
| Original accuracy:            | 70.0%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 71.43% |
| Average perturbed word %:     | 13.74% |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 111.0  |
+-------------------------------+--------+


In [98]:
!nvidia-smi

Sun Apr 14 09:55:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.86.10              Driver Version: 535.86.10    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0              32W /  70W |    651MiB / 15360MiB |     18%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--